# Output Formatters Tutorial

---

### To Get the Most Out of this Tutorial
You may read more about the `output_formatter` plugin type <br/>
on the accompanying slides [hosted at this link](https://docs.google.com/presentation/d/1wFbrP4RBdwf6Hx30adD9DN4jNZm1f0hVaHxV--encS0/edit?usp=sharing).

## Exploring Output Formatters With Scripting

We will walk through three examples of existing output formatters to get a better understanding of the existing options.  
That way you can jump right into using these plugins for common use-cases.

### Importing the GeoIPS Interfaces

Since we are walking through this portion of the tutorial as a Python script, 
we need to import every plugin that we plan to use.

We use the convention:
`{interface_type}.get_plugin({plugin_name})` 
to load the plugins at runtime .

In [ ]:
!geoips config create-registries

In [ ]:
import geoips
from geoips import interfaces

# Since we are reading abi data in a netCDF format, we will use this reader
abi_reader = interfaces.readers.get_plugin("abi_netcdf")

Since we load these plugins dynamically, it can take some extra work to check
their call signatures.  
We can look more closely at the reader plugin by calling:
(this only works in ipython or jupyter notebooks)

In [ ]:
abi_reader?

Here we can see the call signature and docstring from the plugin.  
This should be enough to help you figure out what arguments to pass.

### Reading the ABI Test Data
  
For this test, we will use the data stored in the `GEOIPS_TESTDATA_DIR` directory.
  
We can access the current GeoIPS environment variables by calling:  
`geoips.filenames.base_paths.PATHS`  
which returns a dictionary with the environment variables.

In [ ]:
from glob import glob
from geoips.filenames.base_paths import PATHS as GPATHS
GEOIPS_TESTDATA_DIR = GPATHS['GEOIPS_TESTDATA_DIR']
in_fpath = f"{GEOIPS_TESTDATA_DIR}/test_data_abi/data/goes16_20200918_1950/*"
in_fpaths = glob(in_fpath)

We want to restrict our output to just the CONUS sector

In [ ]:
%%bash
find $HOME -path "*/plugins/*" -type d -name ".ipynb_checkpoints" -exec rm -rf {} +
geoips config create-registries

In [ ]:
conus = interfaces.sectors.get_plugin("conus")

First we need to read the abi data for channel 14 Brightness Temperature

In [ ]:
xdict = abi_reader(fnames=in_fpaths, area_def=conus.area_definition, chans=["B14BT"])

# Since we passed an area_def for "conus", that is the name of the Xarray Object we want to call.
conus_xobj = xdict["conus"]

# Let's look at what conus_xobj contains within it:
conus_xobj

### Applying the Interpolator
We want to use Nearest Neighbor resampling for this dataset, so we will use the interp_nearest interpolator plugin

In [ ]:

interp_nearest = interfaces.interpolators.get_plugin("interp_nearest")

interp_nearest?

In [ ]:
# Now let us apply the Nearest Neighbor interpolation to our data
output_dataset = interp_nearest(conus.area_definition,
                                conus_xobj,
                                None,
                                ["B14BT", "longitude", "latitude"])

# Let's look at our output_dataset
output_dataset

### Applying the Algorithm
Since we will be reading a single channel, we want to use the single_channel algorithm

In [ ]:
import xarray as xr

single_channel_algorithm = interfaces.algorithms.get_plugin("single_channel")

channel_14_bt = output_dataset["B14BT"].data

# Now let us apply the single channel algorithm to our channel 14 Brightness Temperature data
algorithm_output = single_channel_algorithm([channel_14_bt],
                                            output_data_range=[-90.0, 30.0],
                                            input_units="Kelvin",
                                            output_units="celsius")

output_dataset["Infrared"] = xr.DataArray(algorithm_output)

# Let's look at this dataset now that we have created it
output_dataset

### NetCDF Output Formatter

To start off, let's try putting our final data into a netcdf file

In [ ]:
from datetime import datetime, timezone

ncdf_output_formatter = interfaces.output_formatters.get_plugin("netcdf_geoips")

In [ ]:
ncdf_output_formatter?

timestamp = datetime.strftime(datetime.now(timezone.utc), "%Y%m%d%H%M%S")
GEOIPS_OUTDIRS = geoips.filenames.base_paths.PATHS['GEOIPS_OUTDIRS']
out_fpath = f"{GEOIPS_OUTDIRS}/abi_infrared_xarray_test_{timestamp}.nc"

netcdf_output = ncdf_output_formatter(output_dataset,
                                        ["Infrared"],
                                        [out_fpath])
print(f"NetCDF output located at:  {netcdf_output[0]}")

### Non-Annotated Imagery Output Formatter
Taking a step up in complexity, we can call the `imagery_clean` plugin  
which will allow us to write non-annotated imagery

In [ ]:
img_clean_output_formatter = interfaces.output_formatters.get_plugin("imagery_clean")

img_clean_output_formatter?

We need a colormapper to tell matplotlib what colors we want to use

In [ ]:
ir_colormapper = interfaces.colormappers.get_plugin("Infrared")

ir_color_dict = ir_colormapper()

In [ ]:

out_fpath = f"{GEOIPS_OUTDIRS}/abi_infrared_clean_test_{timestamp}.png"

png_clean_output = img_clean_output_formatter(conus.area_definition,
                                             output_dataset,
                                             "Infrared",
                                             [out_fpath],
                                             mpl_colors_info=ir_color_dict)

print(f"Clean Infrared Imagery output located at:  {png_clean_output[0]}")

In [ ]:
from IPython.display import Image

Image(png_clean_output[0])

### Annotated Imagery Output Formatter
Let's start by loading the imagery_annotated plugin

In [ ]:
img_ann_output_formatter = interfaces.output_formatters.get_plugin("imagery_annotated")

img_ann_output_formatter?

We need to tell matplotlib how we want the resulting plot to appear  
using a feature_annotator and a gridline_annotator.  
This time we will use the default versions of these plugins.

In [ ]:
local_feature_annotator = interfaces.feature_annotators.get_plugin("default")

local_gridline_annotator = interfaces.gridline_annotators.get_plugin("default")

Now that we have all the setup completed, we can call our output formatter.  

In [ ]:
out_fpath = f"{GEOIPS_OUTDIRS}/abi_infrared_annotated_test_{timestamp}.png"

# And then we call the plugin
formatter_result_dict={}
png_annotated_output = img_ann_output_formatter(conus.area_definition,
                                           output_dataset,
                                           "Infrared",
                                           [out_fpath],
                                           mpl_colors_info=ir_color_dict,
                                           feature_annotator=local_feature_annotator,
                                           gridline_annotator=local_gridline_annotator,
                                           output_dict=formatter_result_dict)

# Let's print the path to the image on your disk
print(f"Annotated Imagery output located at:  {png_annotated_output}")

In [ ]:
Image(png_annotated_output[0])

# Creating Your Own Output Formatter

First we need to re-do some of the setup work we completed yesterday in the beginner tutorial.

In [ ]:
%%bash

cd $GEOIPS_PACKAGES_DIR

if [[ ! -d "template_basic_plugin" && ! -d "$MY_PKG_NAME" ]]; then
    echo "Cloning template_basic_plugin package"
    git clone --no-tags --single-branch $GEOIPS_REPO_URL/template_basic_plugin.git
else
    echo "Package already exists"
fi

# Rename your package
if [[ ! -d "$MY_PKG_NAME" ]]; then
    echo "Renaming template_basic_plugin to $MY_PKG_NAME"
    mv -v template_basic_plugin/ $MY_PKG_NAME
else
    echo "Package already renamed"
fi

# This will remove references to our upstream repository for safety's sake
cd $MY_PKG_DIR
git remote remove origin 2> /dev/null || true

In [ ]:
%%bash
cd $MY_PKG_DIR

# Rename the package directory
if [[ -d "my_package" ]]; then
    echo "Moving my_package to $MY_PKG_NAME"
    git mv my_package $MY_PKG_NAME
else
    echo "Already moved"
fi

In [ ]:
"""Overwrite cool_plugins' pyproject.toml and README.md with correct contents."""

import os

with open("./updated_files/pyproject.toml", "r") as rf:
    toml_lines = rf.readlines()

with open(f"{os.environ['MY_PKG_DIR']}/pyproject.toml", "w") as wf:
    wf.writelines(toml_lines)

with open("./updated_files/README.md", "r") as rf:
    md_lines = rf.readlines()

with open(f"{os.environ['MY_PKG_DIR']}/README.md", "w") as wf:
    wf.writelines(md_lines)

In [ ]:
%%bash

pip install -e $MY_PKG_DIR

In [ ]:
%%bash

geoips list packages


^^^^^^^^^^^^^^^^^^^  
After running the previous cells, you should see `cool_plugins` in the output directly above this cell!

Otherwise, please ask for help so that we can get you situated!

We will start out by creating a blank output formatter in the cool_plugins package:

In [ ]:
%%bash

mkdir -p $MY_PKG_DIR/$MY_PKG_NAME/plugins/modules/output_formatters

touch $MY_PKG_DIR/$MY_PKG_NAME/plugins/modules/output_formatters/my_netcdf_output.py
touch $MY_PKG_DIR/$MY_PKG_NAME/plugins/modules/output_formatters/__init__.py

Now you should have a blank Python file ready for you to write your great ideas into.  
Open [my_netcdf_output.py](../../cool_plugins/cool_plugins/plugins/modules/output_formatters/my_netcdf_output.py) and now we should give ourselves something to work with...  

By copying the great work that has already been done by others!  

Imagine what sort of output you want to create...  
Do you want to save this data as a bar plot? A blob inside a relational database? Bytes stored as plaintext?  
Or maybe as a beautiful glossy png that you can print out and hang on your wall?  

Let's pretend you all have the same idea, to create a netcdf file with details about the hit 2001 movie, Spy Kids!

Paste the following lines of code into my_netcdf_output.py:  

```
"""My NetCDF output format."""  
import logging  
import xarray    
from geoips.geoips_utils import copy_standard_metadata 
from geoips.plugins.modules.output_formatters.netcdf_xarray import write_xarray_netcdf

LOG = logging.getLogger(__name__)  

interface = "output_formatters"  
family = "xarray_data"  
name = "my_netcdf_output"  
  
def call(xarray_obj, product_names, output_fnames):  
    """Write GeoIPS style NetCDF to disk."""  
    prod_xarray = xarray.Dataset()  
  
    copy_standard_metadata(xarray_obj, prod_xarray)  
    for product_name in product_names:  
        prod_xarray[product_name] = xarray_obj[product_name]  
  
    prod_xarray = prod_xarray.assign_attrs(Starring="Danny Trejo",  
                                           Featuring="Antonio Banderas",  
                                           ProducedBy="Elizabeth Avellán")  
  
    for ncdf_fname in output_fnames:  
        write_xarray_netcdf(prod_xarray, ncdf_fname)  
    return output_fnames  
```

Notice that this code has the following key features of any GeoIPS plugin:  
- A descriptive top-level docstring which tells users what the plugin does
- Top-level imports of third-party and native Python libraries
- A defined logging method **use it!**
- Interface, Family, and Plugin Name
- A call function with its own descriptive docstring
- The call function returns a list of filenames (required by this type of family)

In [ ]:
%%bash
find $HOME -path "*/plugins/*" -type d -name ".ipynb_checkpoints" -exec rm -rf {} +

geoips run single_source $GEOIPS_TESTDATA_DIR/test_data_abi/data/goes16_20200918_1950/* \
    --product_name Infrared \
    --reader_name abi_netcdf \
    --filename_formatter geoips_netcdf_fname \
    --output_formatter my_netcdf_output \
    --resampled_read \
    --logging_level info \
    --sector_list conus  

Look throught the log output for these lines. If you see them, you’ve successfully created a new output formatter!  

```
:Starring = "Danny Trejo" ;
:Featuring = "Antonio Banderas" ;
:ProducedBy = "Elizabeth Avellán" ;
```